<a href="https://colab.research.google.com/github/sindla97/Machine-Learning/blob/main/text_classification_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objective: Text_claasifiacation using distilBERT on IMDB data (50k rows)

1.   Baseline accuracy: LSTM
2.   DistillBERT





In [3]:
#importing library
import pandas as pd
import numpy as np
import tensorflow as tf


In [4]:
#!pip install opendatasets

import opendatasets as od

od.download(
	"https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/data")


Skipping, found downloaded files in "./imdb-dataset-of-50k-movie-reviews" (use force=True to force download)


In [5]:

# reading the XLSX file
file =('/content/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
imdb = pd.read_csv(file)



In [6]:

imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
# removing HTML text
from bs4 import BeautifulSoup
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
imdb['clean_text']  = imdb['review'].apply(lambda x: strip_html(x))

<ipython-input-7-b0ddba9a628a>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train,X_test,y_train,y_test=train_test_split(imdb.clean_text[:10000],imdb.sentiment[:10000],test_size=0.5)

In [10]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_label=le.fit_transform(y_train)
test_label=le.transform(y_test)

#Training using the keras

In [12]:
with tf.device('/cpu:0'):
    train_data = tf.data.Dataset.from_tensor_slices((X_train,train_label))
    valid_data = tf.data.Dataset.from_tensor_slices((X_test,test_label))

    for text,label in train_data.take(10):
      print(text)
      print(label)

tf.Tensor(b'An amazing piece of film that was well-conceived and kept me on the edge of my seat. Brilliantly orchestrated in its timing, and the comedy kicked in exactly when the tension needed a release. The acting was generally well-done (the "Director" should\'ve asked Alec Guinness for acting lessons), and the shot selections were impressive (as in elongating the hall as Billy tries to race to the door in the studio). This movie didn\'t let up since the opening scene...', shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(b"This cartoon was strange, but the story actually had a little more depth and emotion to it than other cartoon movies. We have a girl at a camp with low self esteem and hardly any other friends, except a brother and sister who are just a miserable as she is. She reaches the ultimate low point and when the opportunity arises she literally makes a pact with a devil-like demon. I found this film to be very true to life and just when things couldn'

In [13]:
#!pip install --upgrade tensorflow==2.8.1
print(tf.__version__)


2.8.1


In [14]:
!pip install official

ERROR: Could not find a version that satisfies the requirement official (from versions: none)
ERROR: No matching distribution found for official


In [15]:
import tensorflow_hub as hub
from official.nlp import optimization  # Use this if using TF 2.7 or later
from official.nlp import bert
from official.nlp.bert import tokenization

ModuleNotFoundError: No module named 'official'

In [ ]:
import tensorflow_hub as hub
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", trainable = True )
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def to_feature(text, label, label_list=label_list, max_seq_length=500, tokenizer=tokenizer):
    example = classifier_data_lib.InputExample(guid=None,text_a = text.numpy(),text_b = None,label = label.numpy())
    feature = classifier_data_lib.convert_single_example(0,example,label_list,max_seq_length,tokenizer)
    return (feature.input_ids,feature.input_mask,feature.segment_ids,feature.label_id)

In [19]:
for layer in model.layers:
    layer.trainable = True

In [20]:
max_length = 768
LAYER_DROPOUT = 0.2
LEARNING_RATE = 5e-5
RANDOM_STATE = 42

In [42]:
from tensorflow.keras.layers import Dense, Dropout,Input
from tensorflow.keras.optimizers import Adam

def classification(model,max_length,LAYER_DROPOUT, LEARNING_RATE):
  input_words_ids = tf.keras.layers.Input(shape =(max_seq_length),dtype  = tf.int32, name = "input_word_ids")
  input_mask = tf.keras.layers.Input(shape = (max_seq_length), dtype = tf.int32 , name = "input_mask")
  input_type_ids = tf.keras.layers.Input(shape = (max_seq_length),dtype = tf.int32,name = "input_type_ids")

  pooled_output,sequence_output = bert_layer([input_words_ids,input_mask,input_type_ids])

  cls_token = sequence_output[:, 0, :]
  midlayer = tf.keras.layers.Dense(10, activation='relu')(cls_token)
  midlayer2=tf.keras.layers.Dropout(LAYER_DROPOUT)(midlayer)

  output = tf.keras.layers.Dense(1, activation='softmax')(midlayer2)

    # Define the model
  class_model = tf.keras.Model([input_ids_layer, input_attention_layer], output)
  return class_model



In [43]:
classification(model,max_length,LAYER_DROPOUT, LEARNING_RATE).summary()

AttributeError: module 'tf_keras.api._v1.keras.backend' has no attribute 'run'

In [ ]:
EPOCHS = 6
BATCH_SIZE = 64
NUM_STEPS = len(X_train.index) // BATCH_SIZE

# Train the model
train_history1 = model.fit(
    x = [X_train_ids, X_train_attention],
    y = y_train.to_numpy(),
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    steps_per_epoch = NUM_STEPS,
    validation_data = ([X_valid_ids, X_valid_attention], y_valid.to_numpy()),
    verbose=2
)